In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from sklearn.utils import shuffle
from torch.autograd import Variable

In [2]:
df=pd.read_csv('ratings.csv',names=['userID','movieID','rating','time'])
df.drop('time',axis=1,inplace=True)
users=[k for k,v in df['userID'].value_counts().iteritems() if v>2]
movies=[k for k,v in df['movieID'].value_counts().iteritems() if v>10]
df=df[(df['userID'].isin(users)) & (df['movieID'].isin(movies))]
df=df.sample(frac=1).reset_index(drop=True)
df=df.pivot(index='userID',columns='movieID',values='rating')
df.reset_index(drop=True,inplace=True)
print ('Dataframe size: {}'.format(df.shape))

Dataframe size: (240447, 14277)


In [3]:
train_df=(df.loc[:220000-1])
print ('Train dataframe size: {}'.format(train_df.shape))
test_df=df.loc[230000:].reset_index(drop=True)
print ('Test dataframe size: {}'.format(test_df.shape))
val_df=df.loc[220000:230000-1].reset_index(drop=True)
print ('Validation dataframe size: {}'.format(val_df.shape))

Train dataframe size: (220000, 14277)
Test dataframe size: (10447, 14277)
Validation dataframe size: (10000, 14277)


In [15]:
class Autorec(nn.Module):
    def __init__(self, hidden_size_1, hidden_size_2, dropout, input_size):
        super(Autorec, self).__init__()
        self.input_size=input_size
        self.hidden_size_1=hidden_size_1
        self.hidden_size_2=hidden_size_2
        
        self.encoder_l1=nn.Linear(self.input_size, self.hidden_size_1)
        self.encoder_l2=nn.Linear(self.hidden_size_1, self.hidden_size_2)
        self.decoder_l1=nn.Linear(self.hidden_size_2, self.hidden_size_1)
        self.decoder_l2=nn.Linear(self.hidden_size_1, self.input_size)
        self.drop = nn.Dropout(dropout)
        #self.sigmoid=nn.LogSigmoid()
        
        
    def forward(self, input_ratings):
        #input_ratings=F.normalize(input_ratings)
        enc_out = F.relu(self.encoder_l2(F.relu(self.encoder_l1(input_ratings))))
        enc_out=self.drop(enc_out)
        dec_out = self.decoder_l2(F.relu(self.decoder_l1(enc_out)))
        return dec_out

In [5]:
def train_minibatch(input_ratings, mask, autorec, optimizer):
    optimizer.zero_grad()
    input_ratings=input_ratings.type(torch.cuda.FloatTensor)
    mask=mask.type(torch.cuda.FloatTensor)
    output_ratings=autorec(input_ratings)*mask
    #loss=torch.mean(torch.sum((output_ratings-input_ratings)**2,-1)/torch.sum(mask,-1))
    loss=criterion(input_ratings, output_ratings)
    loss.backward()
    optimizer.step()
    return torch.sqrt(loss)

In [6]:
def validation(input_ratings, mask, autorec):
    with torch.no_grad():
        input_ratings=input_ratings.type(torch.cuda.FloatTensor)
        mask=mask.type(torch.cuda.FloatTensor)
        output_ratings=autorec(input_ratings)*mask
        loss=0
        for i in range(output_ratings.size(0)):
            indices=torch.nonzero(mask[i])
            l=0
            for idx in indices:
                l+=(input_ratings[i][idx]-output_ratings[i][idx])**2
            loss+=l/indices.size(0)
        #loss=torch.mean(torch.sum((output_ratings-input_ratings)**2,-1)/torch.sum(mask,-1))
        return (torch.sqrt(loss/mask.size(0))).item()

In [16]:
autorec=Autorec(hidden_size_1=256, hidden_size_2=128, dropout=0.5, input_size=train_df.shape[1])
optimizer=optim.Adam(autorec.parameters())
device=torch.device('cuda')
criterion=nn.MSELoss()
autorec=autorec.to(device)

In [9]:
val_mask=val_df.copy()
val_mask[~val_mask.isnull()] = 1  # not nan
val_mask[val_mask.isnull()] = 0   # nan
val_df[val_df.isnull()] = 0   # nan

In [10]:
input_users_val=torch.from_numpy(val_df.values).to(device).detach()
mask_val=torch.from_numpy(val_mask.values).to(device).detach()

In [ ]:
checkpoint = torch.load('model.pth')
autorec.load_state_dict(checkpoint['autorec_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
autorec.eval()

In [ ]:
num_batches=5
#val_benchmark=10

counter=0

for batch in range(0,num_batches):
    running_loss=0
    counter=0
    #train_df = shuffle(train_df)
    while counter <train_df.shape[0]:
        tdf=(train_df.loc[counter:counter+99].copy())
        train_mask=tdf.copy()
        train_mask[~train_mask.isnull()] = 1  # not nan
        train_mask[train_mask.isnull()] = 0   # nan
        tdf[tdf.isnull()] = 0
        
        assert tdf.shape[0]==100
        input_users=Variable(torch.from_numpy(tdf.values)).to(device)
        input_mask=torch.from_numpy(train_mask.values).to(device)
        loss=train_minibatch(input_users, input_mask, autorec, optimizer)
        running_loss+=loss.item()
        
        if (counter+100)%44000==0:
            val_rmse=validation(input_users_val, mask_val, autorec)
            print ('Batch: {} | Step: {}/{} | Training Loss: {} | Validation RMSE: {}'.format(batch+1,int((counter+100)/44000),5,running_loss,round(val_rmse,4) ))
            running_loss=0
            if(val_rmse<val_benchmark):
                print ('%---Saving the model---%')
                torch.save({
                    'step':counter+1,
                    'autorec_state_dict': autorec.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'batch':batch,
                    'loss':val_rmse
                   },'model_autorec.pth')
                val_benchmark=val_rmse
        counter=counter+100

Batch: 1 | Step: 1/5 | Training Loss: 8437.022769927979 | Validation RMSE: 0.9937
%---Saving the model---%
Batch: 1 | Step: 2/5 | Training Loss: 8524.698258399963 | Validation RMSE: 0.998
Batch: 1 | Step: 3/5 | Training Loss: 8436.994282722473 | Validation RMSE: 0.998
Batch: 1 | Step: 4/5 | Training Loss: 8234.050210952759 | Validation RMSE: 0.9949
Batch: 1 | Step: 5/5 | Training Loss: 8325.704956054688 | Validation RMSE: 0.999


In [10]:
test_mask=test_df.copy()
test_mask[~test_mask.isnull()] = 1  # not nan
test_mask[test_mask.isnull()] = 0   # nan
test_df[test_df.isnull()] = 0   # nan

In [ ]:
input_users_test=torch.from_numpy(test_df.values).to(device).detach()
mask_test=torch.from_numpy(test_mask.values).to(device).detach()
#print ('RMSE: {}, Precision@10: {}'.format(validation(input_users_test, mask_test, autorec)))

In [12]:
rmse,prec=validation(input_users_test, mask_test, autorec)
print ('RMSE: {}, Precision@10: {}'.format(rmse,prec))

RMSE: 0.8058728485650766, Precision@10: 0.8329706202393907


In [26]:
train_df.shape

(219000, 14277)